I am trying to reproduce [pyannote.audio](https://github.com/pyannote/pyannote-audio) library.

In [3]:
# Clone the repository to access .yml files.

!git clone https://github.com/pyannote/pyannote-audio.git

Cloning into 'pyannote-audio'...
remote: Counting objects: 6916, done.
remote: Compressing objects: 100% (108/108), done.
remote: Total 6916 (delta 83), reused 118 (delta 57), pack-reused 6751
Receiving objects: 100% (6916/6916), 3.35 MiB | 3.24 MiB/s, done.
Resolving deltas: 100% (4385/4385), done.
Checking connectivity... done.


In [4]:
!python --version

Python 3.6.5 :: Anaconda custom (64-bit)


In [5]:
# Read the config.yml file to reproduce exact result. 

!cat ./pyannote-audio/tutorials/change-detection/config.yml

# The MIT License (MIT)
#
# Copyright (c) 2018 CNRS
#
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWIS

In [6]:
from pyannote.database import get_protocol

In this notebook, I will use AMI Corpus.

In [ ]:
# To get shell script to download AMI Corpus. Also, you can create new script from
# http://groups.inf.ed.ac.uk/ami/download/ Please select all meetings and headset
# mix for this example.


!wget http://groups.inf.ed.ac.uk/ami/download/temp/amiBuild-9938-Thu-Jul-19-2018.wget.sh

In [ ]:
# Give permission to AMI Corpus script

!chmod +x amiBuild-9938-Thu-Jul-19-2018.wget.sh

In [ ]:
# Run the shell script to download datas.

! ./amiBuild-9938-Thu-Jul-19-2018.wget.sh

In [ ]:
mkdir .pyannote

In [ ]:
cd ..

In [7]:
# Create new folder to create db.yml inside of that folder.

! cat ./.pyannote/db.yml | grep AMI


AMI: /home/herdogan/Desktop/SpChangeDetect/amicorpus/*/audio/{uri}.wav


I am trying to create db.yml via cat command, however, with Google Colab, I can not do that. So that, I just download this file from my Google Drive. To create, you own db.yml, you can use any text editor. We should write the directory of AMI Corpus. In this case, it is  **AMI: /home/herdogan/Desktop/SpChangeDetect/pyannote-audio/amicorpus/*/audio/{uri}.wav**


In [8]:
!cat /home/herdogan/Desktop/SpChangeDetect/.pyannote/db.yml | grep AMI 

AMI: /home/herdogan/Desktop/SpChangeDetect/amicorpus/*/audio/{uri}.wav


In [9]:
cd /home/herdogan/Desktop/SpChangeDetect/pyannote-audio

/home/herdogan/Desktop/SpChangeDetect/pyannote-audio


In [10]:
!ls

CHANGELOG.md  MANIFEST.in  README.md  setup.cfg  tutorials
LICENSE       pyannote	   scripts    setup.py	 versioneer.py


Now, time for the tutorials. In this notebook, I will provide codes for training and validation. For further information, please check the [repository.](github.com/pyannote/pyannote-audio/tree/master/tutorials)

## **Feature Extraction**

In [11]:
!cat tutorials/feature-extraction/config.yml

# The MIT License (MIT)
#
# Copyright (c) 2018 CNRS
#
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWIS

In [12]:
!pyannote-speech-feature tutorials/feature-extraction AMI.SpeakerDiarization.MixHeadset

Development set: 21it [01:15,  3.61s/it]
Test set: 22it [01:33,  4.27s/it]
Training set: 115it [07:53,  4.12s/it]


In [13]:
!ls tutorials/feature-extraction/AMI | head -n 5

EN2001a.Mix-Headset.npy
EN2001b.Mix-Headset.npy
EN2001d.Mix-Headset.npy
EN2001e.Mix-Headset.npy
EN2002b.Mix-Headset.npy


## *Speech activity detection*

In [14]:
cat tutorials/speech-activity-detection/config.yml

# The MIT License (MIT)
#
# Copyright (c) 2018 CNRS
#
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWIS

If you want to use tensorboard, you can check [this](https://stackoverflow.com/questions/47818822/can-i-use-tensorboard-with-google-colab)

In [15]:
# Training.

!pyannote-speech-detection train tutorials/speech-activity-detection AMI.SpeakerDiarization.MixHeadset

Training set: 115it [00:01, 98.54it/s]
Training set: 115it [00:15,  7.22it/s]
Auto LR:   0%|         | 2/500 [00:02<11:12,  1.35s/it, loss=0.532, lr=1.04e-06]^C


In [ ]:
# Validation.

!pyannote-speech-detection validate tutorials/speech-activity-detection/train/AMI.SpeakerDiarization.MixHeadset.train AMI.SpeakerDiarization.MixHeadset

## *Speaker Change Detection*

Yes. /path/to/scd should be replaced by /absolute/path/to/tutorials/pipeline/scd.

https://github.com/pyannote/pyannote-audio/issues/105

In [16]:
cat tutorials/change-detection/config.yml

# The MIT License (MIT)
#
# Copyright (c) 2018 CNRS
#
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWIS

In [ ]:
# !pyannote-speech-detection -h

In [ ]:
# Training.

!pyannote-speech-detection train --to=5  tutorials/change-detection AMI.SpeakerDiarization.MixHeadset

In [ ]:
# Validation

!pyannote-change-detection validate --purity=0.8 --gpu  --to=5 tutorials/change-detection/train/AMI.SpeakerDiarization.MixHeadset.train AMI.SpeakerDiarization.MixHeadset

In [ ]:
mkdir raw_scores

In [ ]:
!pyannote-change-detection apply tutorials/change-detection/train/AMI.SpeakerDiarization.MixHeadset.train/weights/0005.pt   AMI.SpeakerDiarization.MixHeadset raw_scores

In [ ]:
!pyannote-change-detection apply tutorials/change-detection/train/AMI.SpeakerDiarization.MixHeadset.train/weights/0005.pt   AMI.SpeakerDiarization.MixHeadset path/to/scd

In [1]:
#AMI Protocol
from pyannote.database import get_protocol
protocol = get_protocol('AMI.SpeakerDiarization.MixHeadset')

In [2]:
# Precomputed Scores
from pyannote.audio.features import Precomputed
precomputed = Precomputed('raw_scores')

ValueError: Either directory /home/herdogan/Desktop/SpChangeDetect/raw_scores does not exist or it does not contain precomputed features. In case it exists and this was done on purpose, please provide both `sliding_window` and `dimension` parameters when instantianting `Precomputed`.

In [ ]:
precomputed

In [ ]:
precomputed.shape

In [ ]:
# Peak Detection
from pyannote.audio.signal import Peak
peak = Peak(alpha=0.5, min_duration=1.0, log_scale=True)

In [ ]:
# Evaluation Metric
from pyannote.metrics.diarization import DiarizationPurityCoverageFMeasure
metric = DiarizationPurityCoverageFMeasure()

In [ ]:
# Loop on Test Files
from pyannote.database import get_annotated
for test_file in protocol.test():
    print (test_file)
    # load reference annotation
    reference = test_file['annotation']
    uem = get_annotated(test_file)

    # load precomputed change scores as pyannote.core.SlidingWindowFeature
    scd_scores = precomputed(test_file)

    # binarize scores to obtain speech regions as pyannote.core.Timeline
    hypothesis = peak.apply(scd_scores, dimension=1)

    # evaluate speech activity detection
    metric(reference, hypothesis.to_annotation(), uem=uem)

purity, coverage, fmeasure = metric.compute_metrics()
print(f'Purity = {100*purity:.1f}% / Coverage = {100*coverage:.1f}%')